In [ ]:
import numpy as np
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_16_FT_WritingPrompts_full.xlsx'
df = pd.read_excel(file_path, sheet_name ='Sheet1', engine = 'openpyxl')

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Define columns for prompts and responses
prompts = df['Prompts']
response_1 = df['Response_1']
response_2 = df['Response_2']
response_3 = df['Response_3']

In [ ]:
def keyword_match(question, answer):
    question_tokens = set(word_tokenize(question.lower())) - set(stopwords.words('english'))
    answer_tokens = set(word_tokenize(answer.lower())) - set(stopwords.words('english'))
    overlap = question_tokens & answer_tokens
    return len(overlap) / len(question_tokens)

In [ ]:
# Initialize a list to store average similarity scores and detect functionality of responses
Functionalities = []
# Iterate through each prompt and its three responses
for prompt, resp1, resp2, resp3 in zip(prompts, response_1, response_2, response_3):
    tokens_p = prompt.split()
    resp1 = str(resp1)
    resp2 = str(resp2)
    resp3 = str(resp3)
    tokens_1 = resp1.split()
    tokens_2 = resp2.split()
    tokens_3 = resp3.split()

    total_tokens_p = len(tokens_p)
    unique_tokens_1 = len(set(tokens_1))
    total_tokens_1 = len(tokens_1)
    unique_tokens_2 = len(set(tokens_2))
    total_tokens_2 = len(tokens_2)
    unique_tokens_3 = len(set(tokens_3))
    total_tokens_3 = len(tokens_3)
    lexical_diversity_1 = unique_tokens_1 / total_tokens_1
    lexical_diversity_2 = unique_tokens_2 / total_tokens_2
    lexical_diversity_3 = unique_tokens_3 / total_tokens_3

    # Dependency parsing for syntactic complexity
    doc_1 = nlp(resp1)
    dependency_count_1 = len([token for token in doc_1 if token.dep_ != "ROOT"])
    doc_2 = nlp(resp2)
    dependency_count_2 = len([token for token in doc_2 if token.dep_ != "ROOT"])
    doc_3 = nlp(resp3)
    dependency_count_3 = len([token for token in doc_3 if token.dep_ != "ROOT"])

    match_1 = keyword_match(prompt, resp1)
    match_2 = keyword_match(prompt, resp2)
    match_3 = keyword_match(prompt, resp3)

    func = 0

    if (match_1 >= 0.10) and (dependency_count_1 >= 150) and (lexical_diversity_1 >= 0.5):
      func += 1
    if (match_2 >= 0.10) and (dependency_count_2 >= 150) and (lexical_diversity_2 >= 0.5):
      func += 1
    if (match_3 >= 0.10) and (dependency_count_3 >= 150) and (lexical_diversity_3 >= 0.5):
      func += 1

    # Calculate the average similarity and functionality
    Functionalities.append(func)

In [ ]:
df['Functionality'] = Functionalities

In [ ]:
# Save the updated DataFrame back to Google Drive
updated_file_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_16_FT_WritingPrompts_full.xlsx'
df.to_excel(updated_file_path, index=False, engine='openpyxl')